In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.sql import SparkSession, SQLContext, Row
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

from pyspark.ml.evaluation import MulticlassClassificationEvaluator #BinaryClassificationEvaluator

import numpy as np
import pandas as pd

In [2]:
spark = SparkSession.builder.appName("dataset").getOrCreate()
data = spark.read.csv("refined", header=True).cache()
dataset = data.select(data.Review, data.Category).cache()
dataset.show()


+--------------------+--------------+
|              Review|      Category|
+--------------------+--------------+
|        nao funciona| Instabilidade|
|editei a nota poi...|        Elogio|
|trava muito  nao ...| Instabilidade|
|nao consigo entra...| Instabilidade|
|               otimo|        Elogio|
|atualizou o app a...| Instabilidade|
|depois dessa atua...| Instabilidade|
|o aplicativo trav...| Instabilidade|
|login fica carreg...| Instabilidade|
|nao consigo fazer...| Instabilidade|
|nao consigo abrir...| Instabilidade|
|nao e mais possiv...|Funcionalidade|
|nao funciona nao ...| Instabilidade|
|horrivel  toda ve...| Instabilidade|
|atualizacao ficou...| Instabilidade|
|depois da ultima ...| Instabilidade|
|o app esta com ba...| Instabilidade|
|nao dar para muda...| Instabilidade|
|depois da ultima ...| Instabilidade|
|parabens net  mai...| Instabilidade|
+--------------------+--------------+
only showing top 20 rows



In [3]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

def valueToCategory(value):
    if value == 'Instabilidade':
        return 0

    elif value == 'Elogio':
        return 1

    elif value == 'Funcionalidade':
        return 2

# NOTE: it seems that calls to udf() must be after SparkContext() is called
#udfValueToCategory = udf(valueToCategory, StringType())
#dataset_novo = dataset.withColumn("label", udfValueToCategory("Category"))
#dataset = dataset.select(df.Review, df.label).cache() ## Removendo o campo category
#dataset_novo.show()

from pyspark.sql.types import IntegerType

dataset_novo = udf(lambda z: valueToCategory(z), IntegerType())
dataset_e = dataset.select('Category',dataset_novo('Category').alias('label'),'Review')
dataset_e.show()

+--------------+-----+--------------------+
|      Category|label|              Review|
+--------------+-----+--------------------+
| Instabilidade|    0|        nao funciona|
|        Elogio|    1|editei a nota poi...|
| Instabilidade|    0|trava muito  nao ...|
| Instabilidade|    0|nao consigo entra...|
|        Elogio|    1|               otimo|
| Instabilidade|    0|atualizou o app a...|
| Instabilidade|    0|depois dessa atua...|
| Instabilidade|    0|o aplicativo trav...|
| Instabilidade|    0|login fica carreg...|
| Instabilidade|    0|nao consigo fazer...|
| Instabilidade|    0|nao consigo abrir...|
|Funcionalidade|    2|nao e mais possiv...|
| Instabilidade|    0|nao funciona nao ...|
| Instabilidade|    0|horrivel  toda ve...|
| Instabilidade|    0|atualizacao ficou...|
| Instabilidade|    0|depois da ultima ...|
| Instabilidade|    0|o app esta com ba...|
| Instabilidade|    0|nao dar para muda...|
| Instabilidade|    0|depois da ultima ...|
| Instabilidade|    0|parabens n

In [4]:
dataset_e.schema

StructType(List(StructField(Category,StringType,true),StructField(label,IntegerType,true),StructField(Review,StringType,true)))

### Vetorizando

In [5]:
regex_tokenizer = RegexTokenizer(inputCol="Review", outputCol="words", pattern="\\W")
tokenized = regex_tokenizer.transform(dataset_e)

In [6]:
tokenized.show()

+--------------+-----+--------------------+--------------------+
|      Category|label|              Review|               words|
+--------------+-----+--------------------+--------------------+
| Instabilidade|    0|        nao funciona|     [nao, funciona]|
|        Elogio|    1|editei a nota poi...|[editei, a, nota,...|
| Instabilidade|    0|trava muito  nao ...|[trava, muito, na...|
| Instabilidade|    0|nao consigo entra...|[nao, consigo, en...|
|        Elogio|    1|               otimo|             [otimo]|
| Instabilidade|    0|atualizou o app a...|[atualizou, o, ap...|
| Instabilidade|    0|depois dessa atua...|[depois, dessa, a...|
| Instabilidade|    0|o aplicativo trav...|[o, aplicativo, t...|
| Instabilidade|    0|login fica carreg...|[login, fica, car...|
| Instabilidade|    0|nao consigo fazer...|[nao, consigo, fa...|
| Instabilidade|    0|nao consigo abrir...|[nao, consigo, ab...|
|Funcionalidade|    2|nao e mais possiv...|[nao, e, mais, po...|
| Instabilidade|    0|nao

In [7]:
stop_words = ["to","of","it","I","the","is", "up", "has", "for", "have", "yet", "I've", "a", "as", "this"]
words_remover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(stop_words)
removed = words_remover.transform(tokenized)

In [8]:
removed.show(5)

+-------------+-----+--------------------+--------------------+--------------------+
|     Category|label|              Review|               words|            filtered|
+-------------+-----+--------------------+--------------------+--------------------+
|Instabilidade|    0|        nao funciona|     [nao, funciona]|     [nao, funciona]|
|       Elogio|    1|editei a nota poi...|[editei, a, nota,...|[editei, nota, po...|
|Instabilidade|    0|trava muito  nao ...|[trava, muito, na...|[trava, muito, na...|
|Instabilidade|    0|nao consigo entra...|[nao, consigo, en...|[nao, consigo, en...|
|       Elogio|    1|               otimo|             [otimo]|             [otimo]|
+-------------+-----+--------------------+--------------------+--------------------+
only showing top 5 rows



In [9]:
hashing_tf = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=10000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5)

pipeline = Pipeline(stages=[regex_tokenizer, words_remover, hashing_tf, idf])

In [10]:
pipeline_model = pipeline.fit(dataset_e)
data_text = pipeline_model.transform(dataset_e) 

In [11]:
data_text.show(1)

+-------------+-----+------------+---------------+---------------+--------------------+--------------------+
|     Category|label|      Review|          words|       filtered|         rawFeatures|            features|
+-------------+-----+------------+---------------+---------------+--------------------+--------------------+
|Instabilidade|    0|nao funciona|[nao, funciona]|[nao, funciona]|(10000,[787,4980]...|(10000,[787,4980]...|
+-------------+-----+------------+---------------+---------------+--------------------+--------------------+
only showing top 1 row



In [12]:
(train, test) = data_text.randomSplit([0.7, 0.3], seed=10)

In [13]:
classifier = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0, family="multinomial")#era binomial
model = classifier.fit(train)

In [14]:
predictions = model.transform(test)
predictions.select(predictions.label, predictions.prediction).show(20)

+-----+----------+
|label|prediction|
+-----+----------+
|    1|       0.0|
|    1|       1.0|
|    1|       0.0|
|    1|       0.0|
|    1|       1.0|
|    1|       0.0|
|    1|       0.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       0.0|
|    1|       0.0|
|    1|       0.0|
|    1|       0.0|
|    1|       1.0|
|    1|       1.0|
|    1|       0.0|
|    1|       0.0|
|    1|       1.0|
|    1|       1.0|
+-----+----------+
only showing top 20 rows



In [15]:
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction')#BinaryClassificationEvaluator(rawPredictionCol="prediction")
evaluator.evaluate(predictions)

0.8207774308172714

In [16]:
vector = np.array(predictions.select("label", "prediction").collect())
pd.crosstab(vector[:,0], vector[:,1])

col_0,0.0,1.0,2.0
row_0,,,
0.0,110,1,2
1.0,14,12,0
2.0,14,0,32
